In [1]:
import pandas as pd
import numpy as np


In [44]:
train = pd.read_csv('C:\Users\jyots\Music\NovoEd\CRIME_DATA.csv')
HI = pd.read_csv('C:\Users\jyots\Music\NovoEd\Book1.csv')

In [45]:
train.head(), HI.head()

(             Date         Primary Type  Community Area
 0   1/4/2017 5:30  MOTOR VEHICLE THEFT               1
 1  1/4/2017 13:14    CRIMINAL TRESPASS               1
 2  1/4/2017 15:23              BATTERY               1
 3  1/4/2017 18:30              BATTERY               1
 4  1/4/2017 20:03  MOTOR VEHICLE THEFT               1,    Community Area  HI
 0               1  39
 1               2  46
 2               2  46
 3               2  46
 4               2  46)

In [46]:
#train = train.merge(HI,on='Community Area',how="inner")
train.head()

,Date,Primary Type,Community Area,HI
0,1/4/2017 5:30,MOTOR VEHICLE THEFT,1,39
1,1/4/2017 13:14,CRIMINAL TRESPASS,1,39
2,1/4/2017 15:23,BATTERY,1,39
3,1/4/2017 18:30,BATTERY,1,39
4,1/4/2017 20:03,MOTOR VEHICLE THEFT,1,39


In [47]:
category = train['Primary Type']
prior_prob=category.value_counts(normalize=True)
#prior_prob= pd.DataFrame(prior_prob,columns=['Proir_Prob'])

In [48]:
prior_prob= pd.DataFrame(prior_prob)
prior_prob.columns=['Prior_probs']

In [49]:
train['Hour']= pd.to_datetime(train['Date']).apply(lambda x: x.hour)

In [50]:
#likelihood calculations
likelihood_hour= pd.crosstab(train['Primary Type'],train.Hour).apply(lambda x: x/x.sum(),axis=1)
likelihood_HI = pd.crosstab(train['Primary Type'],train.HI).apply(lambda x: x/x.sum(),axis=1)
likelihood_CAN= pd.crosstab(train['Primary Type'],train['Community Area']).apply(lambda x: x/x.sum(),axis=1)


In [51]:
def classify(row):
    hour= row['Hour']
    HI= row['HI']
    CAN = row['Community Area']
    
    likelihood_hour_sub = likelihood_hour[hour]
    likelihood_HI_sub = likelihood_HI[HI]
    likelihood_CAN_sub = likelihood_CAN[CAN]
    
    #probabilities
    probabilities = prior_prob.join(likelihood_hour_sub,how="inner").join(likelihood_HI_sub,how="inner").join(likelihood_CAN_sub,how="inner")
    probabilities['Probability']= probabilities.prod(axis=1)
    probabilities = probabilities.sort('Probability',ascending=False)
    
    class_label= probabilities.index.values[0]
    return(class_label)


In [82]:
from numpy import random
n= random.randint(0,(len(train)-500))
test = train[n:n+500]


In [83]:
test['ClassLabel'] = test.apply(lambda x: classify(x),axis=1)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [70]:
test.head()

,Date,Primary Type,Community Area,HI,Hour,ClassLabel
60666,10/7/2016 15:45,CRIMINAL DAMAGE,25,73,15,BATTERY
60667,10/7/2016 15:45,CRIMINAL DAMAGE,25,73,15,BATTERY
60668,10/7/2016 15:45,CRIMINAL DAMAGE,25,73,15,BATTERY
60669,10/7/2016 15:45,CRIMINAL DAMAGE,25,73,15,BATTERY
60670,10/7/2016 15:45,CRIMINAL DAMAGE,25,73,15,BATTERY


In [84]:
results = pd.crosstab(test.index,(test['ClassLabel']))
results = pd.DataFrame(results)
results.head()

ClassLabel,ASSAULT,BATTERY,OTHER OFFENSE
row_0,,,
181072,0,1,0
181073,0,1,0
181074,0,1,0
181075,0,1,0
181076,0,1,0


In [85]:
category= prior_prob.index.values
predictions= pd.DataFrame(index=test.index,columns=category)
predictions= predictions.fillna(0)
predictions= (predictions+results).fillna(0)

In [80]:
predictions.to_csv('C:\Users\jyots\Music/predictions.csv')

In [86]:
predictions.head()

,ARSON,ASSAULT,BATTERY,BURGLARY,CONCEALED CARRY LICENSE VIOLATION,CRIM SEXUAL ASSAULT,CRIMINAL DAMAGE,CRIMINAL TRESPASS,DECEPTIVE PRACTICE,GAMBLING,...,OFFENSE INVOLVING CHILDREN,OTHER NARCOTIC VIOLATION,OTHER OFFENSE,PROSTITUTION,PUBLIC PEACE VIOLATION,ROBBERY,SEX OFFENSE,STALKING,THEFT,WEAPONS VIOLATION
181072,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181073,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181074,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181075,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181076,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
